#Mean Shift Clustering

##Overview

<a href='https://en.wikipedia.org/wiki/Mean_shift'>Mean shift clustering</a> is another unsupervised clustering method that we have tested. An advantage of mean shift is that it does not require specifying a number of clusters explicitly beforehand; rather, the algorithm arrives at a certain number of clusters based on a particular radius (or 'bandwidth') parameter which we can call $b$. First, for each point $x_i$ in the dataset, compute the mean of all points from the dataset that lie within a distance $b$ of $x_i$. Then, shift to this mean and repeat the process using that mean as the center. We can repeat this iterative process until convergence, for each $x_i$ in the data. If we do this for each data point, we end up with a cluster center corresponding to each observation, and two points are placed into the same cluster if they end up with cluster centers that are sufficiently close to each other.

This process can be very computationally intensive, because it requires running a separate iterative process for each data point. In addition, it is difficult to intuitively figure out a reasonable value for the bandwidth parameter $b$. To help with this, scikit-learn provides a function called `estimate_bandwidth`, which uses a subsample of the dataset to infer what a reasonable bandwidth would be. In general, a higher bandwidth or radius would cause the number of clusters to be smaller because more points would end up being placed into the same cluster, while a lower bandwidth leads to a larger number of smaller clusters for the same reason.

###Summary of Findings

Ultimately, this model was not very effective, and we figured this out easily by just using it to cluster the training data. (It was apparent that the model was not giving a reasonable output, even without comparing it to the Supreme Court Database topic labels or trying it on the test data.) As a result, this notebook is relatively bare; we just load in the training data for nouns (which were consistently the best part of speech for other models) and try clustering the data using a mean shift for several different parameter values. Since all parameter choices gave useless results, we gave up and did not move further with this.

A detailed description of why the model output was not useful is provided below.

In [1]:
%matplotlib inline
import numpy as np
import sklearn.cluster
import sklearn.feature_extraction
import matplotlib as mpl
import matplotlib.cm as cm
import matplotlib.pyplot as plt
import itertools
from sklearn.cross_validation import KFold
import seaborn as sns
sns.set_style("darkgrid")
sns.set_context("poster")

First, we load in the training data for nouns and apply a TF-IDF transform (which is described in detail elsewhere in our project) to the data, to place a higher importance on words that are not extremely common.

In [2]:
%%time
noun_train_mat = np.loadtxt("noun_train_mat.csv", delimiter = ",")
# use TF-IDF to scale each document's vector to have norm 1 and place a lower weight on very common words
tf_idf_fit = sklearn.feature_extraction.text.TfidfTransformer().fit(noun_train_mat)
noun_train_mat = tf_idf_fit.transform(noun_train_mat).toarray()

Wall time: 32.5 s


Now, we use the scikit-learn `estimate_bandwidth` function with a subsample of 10% of the training data observations, to estimate what the bandwidth parameter for the mean shift model should be.

In [3]:
%%time
n_samples = len(noun_train_mat) / 10
bandwidth = sklearn.cluster.estimate_bandwidth(noun_train_mat, n_samples=n_samples)
print bandwidth

1.3806139741
Wall time: 325 ms


Using this bandwidth choice, we can fit a mean shift model and use it to cluster the data.

In [4]:
%%time
fit = sklearn.cluster.MeanShift(bandwidth=bandwidth).fit(noun_train_mat)

Wall time: 8min 37s


In [5]:
num_clusters = max(fit.predict(noun_train_mat)) + 1
print 'Number of clusters:', num_clusters
cluster_counts = [sum([x==cluster for x in fit.predict(noun_train_mat)]) for cluster in range(num_clusters)]
print 'Number of documents per cluster:', cluster_counts

Number of clusters: 1
Number of documents per cluster: [1087]


So, if we use the optimal bandwidth parameter from `estimate_bandwidth`, which was around 1.38 when we ran this, all of the observations end up getting placed into a single cluster. Obviously, this is not particularly informative or useful.

To improve upon this result, we decided to try fitting mean shift models with various different choices for the bandwidth, ranging from 0.1 to 100. (As a side note, we know that a bandwidth larger than 1.38 should also cause all the data to be placed into a single cluster based on the explanation at the beginning, but we decided that it was worth testing those high bandwidth choices regardless.)

In [6]:
%%time
bandwidths = [0.1, 0.25, 0.5, 0.75, 0.9, 0.95, 1, 10, 100]
fits = []
for bandwidth in bandwidths:
    print 'Testing with bandwidth=%.2f' % bandwidth
    fits = fits + [sklearn.cluster.MeanShift(bandwidth=bandwidth).fit(noun_train_mat)]

Testing with bandwidth=0.10
Testing with bandwidth=0.25
Testing with bandwidth=0.50
Testing with bandwidth=0.75
Testing with bandwidth=0.90
Testing with bandwidth=0.95
Testing with bandwidth=1.00
Testing with bandwidth=10.00
Testing with bandwidth=100.00
Wall time: 27min 54s


Now, we can output the results of those mean shift algorithms.

In [7]:
%%time
for bandwidth,fit in zip(bandwidths, fits):
    print 'BANDWIDTH:', bandwidth
    predictions = fit.predict(noun_train_mat)
    num_clusters = max(predictions) + 1
    print 'Number of clusters:', num_clusters
    print 'Number of documents per cluster:', [sum([x==cluster for x in predictions]) for cluster in range(num_clusters)]
    print

BANDWIDTH: 0.1
Number of clusters: 1087
Number of documents per cluster: [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,

We can see here (without even looking at the Supreme Court Database data or test data) that there is no parameter choice that produces a reasonable result. The low bandwidth values like 0.1 and 0.75 cause the algorithm to assign nearly every individual point to its own cluster, which is not very meaningful. At the other extreme, a bandwidth of 1 or higher causes nearly all (or all) points to be placed into a single massive cluster, which is also not very useful. Bandwidth values of 0.9 or 0.95 seem like they produce one reasonably sized cluster, and many clusters that are far too small.

For this reason, we decided not to pursue this model further. Based on the characteristics of this particular data set, a mean shift method would not be able to produce very meaningful or useful clusters, so it is not worth doing a more in-depth analysis, especially since the mean shift algorithm is so computationally intensive to run.